In [1]:
!pip install category_encoders
import pandas as pd
import pickle
import  category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from category_encoders import *
import  numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#Downloading the data, removing NAs and shuffling (the data will not be shuffled on crossfold)
url = "https://raw.githubusercontent.com/joao-l-rossi/LHL---midterm/main/data/Flights_TRAIN.csv"
dataset = pd.read_csv(url)
dataset = dataset.dropna()
dataset = dataset.sample(frac=1, random_state=1234).reset_index(drop=True) #Edit "frac" to try witha  subsample of the data
dataset.shape

(143046, 29)

In [3]:
#columns to keep and to encode
cols_to_encode = ["origin", "dest","op_unique_carrier"]
cols_X = ['dep_dayofweek','op_unique_carrier', 'origin', 'dest',
       'arr_Fog', 'arr_Snow', 'arr_Cold', 'arr_Storm', 'arr_Hail',
          'arr_Precipitation', 'crs_dep_time', 'crs_arr_time',
          'flights_origin', 'flights_dest']


In [4]:
# Setting up X and y
X = dataset[cols_X]
y = dataset["arr_delay"]

In [5]:
#Splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1234)

In [6]:
#Reseting indexes
X_train = X_train.reset_index(drop=True)     
y_train = y_train.reset_index(drop=True)

In [7]:
#Creating encode object
encoder = TargetEncoder(cols=cols_to_encode)

In [8]:
#Building stratified encoding (to minimize leakage among bins)
X_train_fitted = X_train.iloc[0:0,:]
kf = KFold(n_splits = 5, shuffle=False)
for tr_ind, val_ind in kf.split(X_train, y_train):
    encoder.fit(X_train.loc[tr_ind], y_train.loc[tr_ind])
    X_train_fitted = X_train_fitted.append(encoder.transform(X_train.loc[val_ind]))

In [9]:
#Encoding test
encoder.fit(X_train, y_train)
X_test_fitted = encoder.transform(X_test)

#### Baseline Model

In [10]:
mod = RandomForestRegressor()
mod.fit(X_train_fitted,y_train)

RandomForestRegressor()

In [11]:
print(f"R2: {mod.score(X_test_fitted,y_test)}")
y_pred = mod.predict(X_test_fitted)
print(f"MSE: {metrics.mean_squared_error(y_test,y_pred)}")

R2: 0.09872957407620142
MSE: 1952.6367113421877


In [12]:
# # List of parameters we can calibrate, I have ran differnt combinations of it
# params = {'bootstrap': [True, False],
#           'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
#           'max_features': ['auto', 'sqrt'],
#           'min_samples_leaf': [1, 2, 4],
#           'min_samples_split': [2, 5, 10],
#           'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [13]:
# Final run params
params = {
  'max_features': [0.6, 0.8, 'auto'],
  'max_depth': [5, 10, 30],
  'n_estimators': [300,400]

}

In [14]:
param_comb = 10
mod = RandomForestRegressor()

In [15]:
random_search = RandomizedSearchCV(mod, param_distributions=params, n_iter=param_comb, n_jobs=-1, verbose=10, random_state=1234, cv = kf.split(X_train, y_train))
random_search.fit(X_train_fitted, y_train)
print(random_search.best_score_)
print(random_search.best_estimator_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0.08937711618727165
RandomForestRegressor(max_depth=10, max_features=0.6, n_estimators=400)


In [16]:
# "Best" model
best_est = random_search.best_estimator_
best_est

RandomForestRegressor(max_depth=10, max_features=0.6, n_estimators=400)

In [17]:
print(f"R2: {best_est.score(X_test_fitted,y_test)}")
y_pred = best_est.predict(X_test_fitted)
print(f"MSE: {metrics.mean_squared_error(y_test,y_pred)}")

R2: 0.11361020214247519
MSE: 1920.3972637643399


In [18]:
# Open Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [19]:
#Creating pickle
model_save_name = 'Finalized_model.sav'
path = F"/content/gdrive/My Drive/MidtermLHL/{model_save_name}"
pickle.dump(best_est, open(path, 'wb'))